In [66]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

# For Python2 this have to be done
# from __future__ import division


### Import Toronto data as df,  userJson as df_user

In [83]:
# read csv data
df_toronto_review = pd.read_csv('../data/toronto_reviews.csv')
df_business = pd.read_csv('../data/businesses_final.csv')

In [84]:
# rename business id
df_business.rename(columns={'id': 'business_id'}, inplace=True)

In [85]:
# Drop duplicate values in df_business
df_business_clean = df_business.drop_duplicates(subset ="business_id", 
                     keep = False)

In [86]:
# Rename Month as Day and Day as Month(error occurs on original data)
df_toronto_review=df_toronto_review.rename(columns = {'Month':'new_Day'})
df_toronto_review=df_toronto_review.rename(columns = {'Day':'Month'})
df_toronto_review=df_toronto_review.rename(columns = {'new_Day':'Day'})

In [73]:
# df_toronto_review.shape

(747183, 23)

In [74]:
# df_business_clean.shape

(32103, 17)

In [87]:
# inner join two full dataset
df = pd.merge(df_toronto_review, df_business_clean, on='business_id', how='inner')

In [76]:
# df.shape

(747015, 39)

In [88]:
# merge the date and turn it into string
df['date'] = pd.to_datetime(df[['Year', 'Month','Day']])
df['date'] = df['date'].astype(str)

In [89]:
# rename the columns to match the algorithm 
df.rename(columns={'rating_x': 'stars','rating_y': 'business_stars','review_text': 'text',}, inplace=True)

In [90]:
# get rid of the rows without user_id
df = df[~df["user_id"].isnull()]

In [91]:
df

,Unnamed: 0_x,Unnamed: 0.1,business_id,friend_count,ghost,img_dsc,img_url,nr,photo_count,stars,...,is_closed,location,name,phone,price,business_stars,review_count_y,transactions,url,date
0,0,0,bSQGCheX1BwvL67Az1OJlA,90,False,[],[],False,231.0,5.0,...,False,"{'address1': '851 Bay Street', 'address2': '',...",Sleep Country,1.416642e+10,$$$$,2.5,3,[],https://www.yelp.com/biz/sleep-country-toronto...,2018-09-05
1,1,1,bSQGCheX1BwvL67Az1OJlA,0,False,[],[],False,9.0,1.0,...,False,"{'address1': '851 Bay Street', 'address2': '',...",Sleep Country,1.416642e+10,$$$$,2.5,3,[],https://www.yelp.com/biz/sleep-country-toronto...,2018-03-06
2,2,2,bSQGCheX1BwvL67Az1OJlA,52,False,"[""Photo of Sleep Country - Toronto, ON, Canada...",['https://s3-media1.fl.yelpcdn.com/bphoto/E_gL...,False,27.0,1.0,...,False,"{'address1': '851 Bay Street', 'address2': '',...",Sleep Country,1.416642e+10,$$$$,2.5,3,[],https://www.yelp.com/biz/sleep-country-toronto...,2016-07-30
3,3,3,Z7JV11GXw_RsGr8Ik_Hsag,0,False,[],[],False,NaN,5.0,...,False,"{'address1': '5 Rowntree Road', 'address2': 'S...",Pest Solution Services,1.416452e+10,NaN,5.0,1,[],https://www.yelp.com/biz/pest-solution-service...,2018-12-16
4,4,4,Z7JV11GXw_RsGr8Ik_Hsag,0,False,[],[],False,NaN,5.0,...,False,"{'address1': '5 Rowntree Road', 'address2': 'S...",Pest Solution Services,1.416452e+10,NaN,5.0,1,[],https://www.yelp.com/biz/pest-solution-service...,2014-08-06
5,5,5,kI7ObOAe-Wvjdtnv-8Cu7w,74,False,[],[],False,NaN,5.0,...,False,"{'address1': '720 Av Spadina', 'address2': '',...",Ccim Central Canada,1.416244e+10,NaN,5.0,1,[],https://www.yelp.com/biz/ccim-central-canada-t...,2015-08-16
6,6,6,Xo1LNzhnwE-ilqsM3ybs9Q,105,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tu7j...,False,325.0,4.0,...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-08-23
7,7,7,Xo1LNzhnwE-ilqsM3ybs9Q,171,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media3.fl.yelpcdn.com/bphoto/h110...,False,307.0,3.0,...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-10-02
8,8,8,Xo1LNzhnwE-ilqsM3ybs9Q,239,False,"['Photo of Happy Lemon - Markham, ON, Canada. ...",['https://s3-media4.fl.yelpcdn.com/bphoto/tS6Y...,False,78.0,5.0,...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-11-06
9,9,9,Xo1LNzhnwE-ilqsM3ybs9Q,10,False,"['Photo of Happy Lemon - Markham, ON, Canada',...",['https://s3-media3.fl.yelpcdn.com/bphoto/rfB0...,False,504.0,4.0,...,False,"{'address1': 'B25 4300 Steeles Avenue E', 'add...",Happy Lemon,1.416855e+10,$,3.5,32,[],https://www.yelp.com/biz/happy-lemon-markham?a...,2016-09-25


### Data Processing

In [65]:
Export = df.to_json(r'..\\data\\Export_TorontoData.json', orient='records')